In [1]:
from pybrain.datasets            import ClassificationDataSet
from pybrain.utilities           import percentError
from pybrain.tools.shortcuts     import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules   import SoftmaxLayer

In [2]:
from pylab import ion, ioff, figure, draw, contourf, clf, show, hold, plot
from scipy import diag, arange, meshgrid, where
from numpy.random import multivariate_normal

In [3]:
means = [(-1,0),(2,4),(3,1)]
cov = [diag([1,1]), diag([0.5,1.2]), diag([1.5,0.7])]
alldata = ClassificationDataSet(2, 1, nb_classes=3)
for n in xrange(400):
    for klass in range(3):
        input = multivariate_normal(means[klass],cov[klass])
        alldata.addSample(input, [klass])

In [4]:
tstdata, trndata = alldata.splitWithProportion( 0.25 )

In [5]:
trndata._convertToOneOfMany( )
tstdata._convertToOneOfMany( )

In [6]:
print "Number of training patterns: ", len(trndata)
print "Input and output dimensions: ", trndata.indim, trndata.outdim
print "First sample (input, target, class):"
print trndata['input'][0], trndata['target'][0], trndata['class'][0]

Number of training patterns:  900
Input and output dimensions:  2 3
First sample (input, target, class):
[-2.31176778 -0.0887695 ] [1 0 0] [ 0.]


In [7]:
fnn = buildNetwork( trndata.indim, 5, trndata.outdim, outclass=SoftmaxLayer )

In [8]:
trainer = BackpropTrainer( fnn, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.01)

In [9]:
ticks = arange(-3.,6.,0.2)
X, Y = meshgrid(ticks, ticks)
# need column vectors in dataset, not arrays
griddata = ClassificationDataSet(2,1, nb_classes=3)
for i in xrange(X.size):
    griddata.addSample([X.ravel()[i],Y.ravel()[i]], [0])
griddata._convertToOneOfMany()  # this is still needed to make the fnn feel comfy

In [13]:
for i in range(100):
    trainer.trainEpochs( 1 )
    trnresult = percentError( trainer.testOnClassData(),
                              trndata['class'] )
    tstresult = percentError( trainer.testOnClassData(
           dataset=tstdata ), tstdata['class'] )

    print "epoch: %4d" % trainer.totalepochs, \
          "  train error: %5.2f%%" % trnresult, \
          "  test error: %5.2f%%" % tstresult
    out = fnn.activateOnDataset(griddata)
    out = out.argmax(axis=1)  # the highest output activation gives the class
    out = out.reshape(X.shape)
    figure(1)
    ioff()  # interactive graphics off
    clf()   # clear the plot
    hold(True) # overplot on
    for c in [0,1,2]:
        here, _ = where(tstdata['class']==c)
        plot(tstdata['input'][here,0],tstdata['input'][here,1],'o')
    if out.max()!=out.min():  # safety check against flat field
        contourf(X, Y, out)   # plot the contour
    ion()   # interactive graphics on
    draw()  # update the plot

Total error: 0.0168018648258
epoch:   21   train error:  5.22%   test error:  6.00%
Total error: 0.0166616323039
epoch:   22   train error:  5.11%   test error:  6.00%
Total error: 0.0166290459511
epoch:   23   train error:  5.22%   test error:  6.33%
Total error: 0.0165837283305
epoch:   24   train error:  5.11%   test error:  6.33%
Total error: 0.0167955459478
epoch:   25   train error:  5.22%   test error:  6.33%
Total error: 0.0167097479878
epoch:   26   train error:  5.22%   test error:  6.33%
Total error: 0.0166041556972
epoch:   27   train error:  5.33%   test error:  7.00%
Total error: 0.0166392783225
epoch:   28   train error:  5.33%   test error:  6.00%
Total error: 0.0165683857563
epoch:   29   train error:  5.33%   test error:  6.67%
Total error: 0.0166059259951
epoch:   30   train error:  5.00%   test error:  6.00%
Total error: 0.0166471386979
epoch:   31   train error:  5.44%   test error:  7.00%
Total error: 0.0166075904549
epoch:   32   train error:  5.22%   test error:

In [14]:
ioff()
show()